In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Extract Features

In [ ]:
%%writefile /content/drive/MyDrive/CODE/KHKT/extract_features.py

import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor
import matplotlib.pyplot as plt
import pandas as pd
import os

# === CẤU HÌNH ===
MODEL_PATH = "/content/drive/MyDrive/CODE/KHKT/checkpoints/segformer_floodnet_epoch15"
TEST_DIR = "/content/drive/MyDrive/CODE/KHKT/FloodNet/test/img"
SAVE_DIR = "/content/drive/MyDrive/CODE/KHKT/FloodNet/test_results"
os.makedirs(SAVE_DIR, exist_ok=True)

# Load processor
try:
    processor = SegformerImageProcessor.from_pretrained(
        MODEL_PATH,
        local_files_only=True
    )
    print("Loaded processor from checkpoint.")
except:
    processor = SegformerImageProcessor.from_pretrained(
        "nvidia/segformer-b0-finetuned-ade-512-512",
        do_reduce_labels=False,
        size={"height": 512, "width": 512}
    )
    print("Loaded default base processor.")

# Label mapping (FloodNet 0–9)
id2label = {
    0: "background",
    1: "building_flooded",
    2: "building_non_flooded",
    3: "road_flooded",
    4: "road_non_flooded",
    5: "water",
    6: "tree",
    7: "vehicle",
    8: "pool",
    9: "grass",
}
label2id = {v: k for k, v in id2label.items()}

# Load model
model = SegformerForSemanticSegmentation.from_pretrained(
    MODEL_PATH,
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    local_files_only=True
)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Color map
id2color = {
    0: (0, 0, 0),
    1: (0, 0, 128),
    2: (0, 0, 255),
    3: (255, 255, 0),
    4: (128, 128, 128),
    5: (0, 255, 255),
    6: (0, 128, 0),
    7: (255, 0, 0),
    8: (128, 0, 0),
    9: (0, 255, 0),
}

# === HÀM INFERENCE ===
def predict_flood_level(image_path):
    # 1. Đọc ảnh
    image = Image.open(image_path).convert("RGB")

    # 2. Preprocess
    inputs = processor(images=image, return_tensors="pt")
    pixel_values = inputs.pixel_values.to(device)

    # 3. Inference
    with torch.no_grad():
        outputs = model(pixel_values)
        logits = outputs.logits  # (1, 10, h, w)

    # 4. Upsample về kích thước ảnh gốc
    upsampled_logits = F.interpolate(
        logits,
        size=image.size[::-1],  # (H, W)
        mode="bilinear",
        align_corners=False
    )
    seg = upsampled_logits.argmax(dim=1)[0].cpu().numpy()  # (H, W)

    # 5. Tính tỉ lệ nước
    flood_classes = [1, 3, 5]
    water_ratio = np.isin(seg, flood_classes).sum() / seg.size


    # 6. Phân mức ngập theo tỉ lệ water
    if water_ratio < 0.05:
        level, level_text = 0, "Không ngập"
    elif water_ratio < 0.1:
        level, level_text = 1, "Ngập nhẹ"
    elif water_ratio < 0.3:
        level, level_text = 2, "Ngập vừa"
    else:
        level, level_text = 3, "Ngập nặng"

    # 7. Tạo mask màu đẹp
    h, w = seg.shape
    mask_colored = np.zeros((h, w, 3), dtype=np.uint8)
    for label, color in id2color.items():
        mask_colored[seg == label] = color

    return {
        "original": image,
        "segmentation": mask_colored,
        "seg_map": seg,
        "water_ratio": water_ratio,
        "flood_level": level,
        "flood_text": level_text
    }

# Các class ngập
FLOOD_CLASSES = [1, 3, 5]  # building_flooded, road_flooded, water

def extract_features(image_path):
    """
    Trích xuất feature để làm input cho mô hình dự báo ngập tương lai.
    """
    result = predict_flood_level(image_path)
    seg_map = result["seg_map"]

    # 1) Các tỷ lệ pixel
    total = seg_map.size
    ratios = {
        "water_ratio": np.mean(seg_map == 5),
        "building_flooded_ratio": np.mean(seg_map == 1),
        "road_flooded_ratio": np.mean(seg_map == 3),
        "overall_flood_ratio": np.mean(np.isin(seg_map, FLOOD_CLASSES)),
        "tree_ratio": np.mean(seg_map == 6),
        "vehicle_ratio": np.mean(seg_map == 7),
        "pool_ratio": np.mean(seg_map == 8),
        "grass_ratio": np.mean(seg_map == 9),
    }

    # 2) Trích xuất feature từ encoder của SegFormer
    img = Image.open(image_path).convert("RGB")

    inputs = processor(images=img, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        last_feat = outputs.hidden_states[-1]     # (1, C, H', W')
        pooled_feat = last_feat.mean(dim=[2, 3])  # => (1, C)
        pooled_feat = pooled_feat.cpu().numpy().flatten()

    # Trả về dictionary + vector feature
    return ratios, pooled_feat

Writing /content/drive/MyDrive/CODE/KHKT/extract_features.py


## Build future dataset

In [ ]:
%%writefile /content/drive/MyDrive/CODE/KHKT/build_synthetic_physics_dataset.py
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys

sys.path.append("/content/drive/MyDrive/CODE/KHKT")
from extract_features import extract_features  # dùng hàm bạn đã có

IMAGE_DIR = "/content/drive/MyDrive/CODE/KHKT/FloodNet/validation/img"

# === HÀM TẠO PROFILE MƯA BẬC 1 (LINEAR) ===
def generate_rain_profile(r0):
    """
    Sinh lượng mưa 1h, 3h, 6h, 12h theo mô hình tuyến tính:
        rain(t) = r0 + slope * t + noise
    Điều kiện:
        - slope random nhỏ để mưa thay đổi không đột ngột
        - noise giới hạn ±1 mm/h
        - không vượt quá r0 ± 10
        - không vượt quá [0, 70]
    """

    # -------- 1. Random slope --------
    # Cho r0 nhỏ → slope nhỏ
    # Cho r0 lớn → slope có thể âm (mưa giảm)
    slope_range = 0.10 + (20 - r0) / 200   # giảm khi r0 lớn
    slope = np.random.uniform(-slope_range, slope_range)

    # -------- 2. Thời gian (giờ) --------
    times = np.array([1, 3, 6, 12], dtype=float)

    # -------- 3. Noise rất nhỏ để profile trông thật --------
    noise = np.random.uniform(-1.0, 1.0, size=4)

    # -------- 4. Tính lượng mưa --------
    r = r0 + slope * times + noise

    # -------- 5. Giới hạn vật lý --------
    r_min = max(r0 - 10.0, 0.0)
    r_max = min(r0 + 10.0, 70.0)
    r = np.clip(r, r_min, r_max)

    return float(r[0]), float(r[1]), float(r[2]), float(r[3])

# === HÀM VẬT LÝ TÍNH TIẾN TRIỂN NGẬP ===

def compute_physics_based_progression(ratios, flood_prev, rain_prev, rain_next, dt_hours):
    """
    Mô phỏng flood progression từ t1 → t2 theo mưa dự báo.
    dt_hours: số giờ trôi qua giữa 2 mốc.
    """
    # 1. Rain average
    rain_avg = (rain_prev + rain_next) / 2.0

    # 2. Rain effect (mm/h -> flood delta)
    if rain_avg < 2:
        rain_effect_per_hour = np.interp(rain_avg, [0, 2], [-0.03, -0.01])
    elif rain_avg < 5:
        rain_effect_per_hour = np.interp(rain_avg, [2, 5], [0.0, 0.02])
    elif rain_avg < 12:
        rain_effect_per_hour = np.interp(rain_avg, [5, 12], [0.02, 0.05])
    elif rain_avg < 20:
        rain_effect_per_hour = np.interp(rain_avg, [12, 20], [0.05, 0.08])
    else:
        rain_effect_per_hour = np.interp(min(rain_avg, 100), [20, 100], [0.08, 0.10])

    # Tổng hiệu ứng mưa theo thời gian dt
    rain_total = rain_effect_per_hour * dt_hours

    drainage_total = -0.05 * dt_hours

    # 3. Flood next = flood_prev + effects
    flood_next = flood_prev + rain_total + drainage_total

    # ép trong [0,1]
    flood_next = float(np.clip(flood_next, 0.0, 1.0))

    return flood_next

# === BUILD DATASET ===
data = []

print("🔧 Đang tạo synthetic_physics_multistep_dataset.pkl ...")

for img_name in tqdm(os.listdir(IMAGE_DIR)):
    if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(IMAGE_DIR, img_name)

    ratios, enc = extract_features(img_path)
    cur_flood = ratios["overall_flood_ratio"]

    # random mưa hiện tại (0..50 mm/h)
    rain_now = float(np.random.uniform(0, 50))

    # tạo profile mưa theo bậc 2
    rain_1h, rain_3h, rain_6h, rain_12h = generate_rain_profile(rain_now)

    # tiến triển ngập theo từng mốc (dùng dạng "chuỗi thời gian")
    flood_now = cur_flood

    # 0h → 1h (dt = 1h)
    flood_1h = compute_physics_based_progression(
        ratios, flood_now, rain_now, rain_1h, dt_hours=1
    )

    # 1h → 3h (dt = 2h)
    flood_3h = compute_physics_based_progression(
        ratios, flood_1h, rain_1h, rain_3h, dt_hours=2
    )

    # 3h → 6h (dt = 3h)
    flood_6h = compute_physics_based_progression(
        ratios, flood_3h, rain_3h, rain_6h, dt_hours=3
    )

    # 6h → 12h (dt = 6h)
    flood_12h = compute_physics_based_progression(
        ratios, flood_6h, rain_6h, rain_12h, dt_hours=6
    )

    row = {
        "filename": img_name,
        "rain_now": rain_now,
        "rain_1h": rain_1h,
        "rain_3h": rain_3h,
        "rain_6h": rain_6h,
        "rain_12h": rain_12h,
        "flood_now": flood_now,
        "flood_1h": flood_1h,
        "flood_3h": flood_3h,
        "flood_6h": flood_6h,
        "flood_12h": flood_12h,
        **ratios,
        "encoder_feature": enc,
    }

    data.append(row)

df = pd.DataFrame(data)
out_path = "/content/drive/MyDrive/CODE/KHKT/synthetic_physics_multistep_dataset.pkl"
df.to_pickle(out_path)

print("🎉 HOÀN TẤT:", out_path)
print(df.head())

Overwriting /content/drive/MyDrive/CODE/KHKT/build_synthetic_physics_dataset.py


In [ ]:
%cd /content/drive/MyDrive/CODE/KHKT
!python build_synthetic_physics_dataset.py

## Train model

In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

DATA_PATH = "/content/drive/MyDrive/CODE/KHKT/synthetic_physics_multistep_dataset.pkl"
df = pd.read_pickle(DATA_PATH)

# tách encoder_feature
enc_mat = np.stack(df["encoder_feature"].values)

semantic_cols = [
    "water_ratio",
    "building_flooded_ratio",
    "road_flooded_ratio",
    "overall_flood_ratio",
    "tree_ratio",
    "vehicle_ratio",
    "pool_ratio",
    "grass_ratio",
    "rain_now",
    "rain_1h",
    "rain_3h",
    "rain_6h",
    "rain_12h",
]

X_sem = df[semantic_cols].values
X = np.concatenate([X_sem, enc_mat], axis=1)

y = df[["flood_1h", "flood_3h", "flood_6h", "flood_12h"]].values

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)
model.fit(X, y)

y_pred = model.predict(X)
mae = mean_absolute_error(y, y_pred)
print("MAE (multi-output):", mae)

out_model = "/content/drive/MyDrive/CODE/KHKT/future_flood_multistep.pkl"
joblib.dump(model, out_model)
print("Đã lưu model:", out_model)

MAE (multi-output): 0.006246772216803885
Đã lưu model: /content/drive/MyDrive/CODE/KHKT/future_flood_multistep.pkl


## Tạo dashboard

In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 118.9 MB/s eta 0:00:00


In [ ]:
%%writefile /content/drive/MyDrive/CODE/KHKT/app.py
import streamlit as st
import numpy as np
import joblib
from PIL import Image
from tempfile import NamedTemporaryFile
import requests
import sys
import os
import math
from io import BytesIO
from difflib import get_close_matches
import unicodedata

# ADD MODULE PATH (để import extract_features.py)
sys.path.append("/content/drive/MyDrive/CODE/KHKT")

# Import segmentation model & feature extractor
from extract_features import extract_features, predict_flood_level

# Load future flood predictor model
predictor_model = joblib.load("/content/drive/MyDrive/CODE/KHKT/future_flood_multistep.pkl")

# WEATHER API KEY
API_KEY = "b3e86069b418794e0effd1c9d29c2466"

# 1. DANH SÁCH TỈNH THÀNH VIỆT NAM + GỢI Ý
VIETNAM_PROVINCES = [
    "An Giang", "Ba Ria - Vung Tau", "Bac Giang", "Bac Kan", "Bac Lieu", "Bac Ninh",
    "Ben Tre", "Binh Dinh", "Binh Duong", "Binh Phuoc", "Binh Thuan",
    "Ca Mau", "Can Tho", "Cao Bang",
    "Da Nang", "Dak Lak", "Dak Nong", "Dien Bien", "Dong Nai", "Dong Thap",
    "Gia Lai", "Ha Giang", "Ha Nam", "Ha Noi", "Ha Tinh",
    "Hai Duong", "Hai Phong", "Hau Giang", "Hoa Binh",
    "Ho Chi Minh City", "HCM",
    "Hung Yen", "Khanh Hoa", "Kien Giang", "Kon Tum",
    "Lai Chau", "Lam Dong", "Lang Son", "Lao Cai", "Long An",
    "Nam Dinh", "Nghe An", "Ninh Binh", "Ninh Thuan",
    "Phu Tho", "Phu Yen", "Quang Binh", "Quang Nam", "Quang Ngai",
    "Quang Ninh", "Quang Tri",
    "Soc Trang", "Son La", "Tay Ninh", "Thai Binh", "Thai Nguyen",
    "Thanh Hoa", "Thua Thien Hue", "Tien Giang", "Tra Vinh",
    "Tuyen Quang", "Vinh Long", "Vinh Phuc",
    "Yen Bai"
]

def normalize_text(s):
    s = s.lower().strip()
    s = unicodedata.normalize('NFD', s)
    return ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')

def suggest_locations(text):
    text_norm = normalize_text(text)
    province_norm_map = {normalize_text(p): p for p in VIETNAM_PROVINCES}
    matches = get_close_matches(text_norm, province_norm_map.keys(), n=5, cutoff=0.4)
    return [province_norm_map[m] for m in matches]


# 2. GEOLOCATION API (OpenWeather)
def get_coordinates(location_name):
    """Chuyển tên tỉnh -> lat, lon có fuzzy fix."""
    name = location_name.strip()
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={name}&limit=5&appid={API_KEY}"
    r = requests.get(url).json()

    if not r:
        # Nếu lỗi -> thử fuzzy suggestion
        suggestions = suggest_locations(name)
        if not suggestions:
            return None, None

        fixed = suggestions[0]
        url2 = f"http://api.openweathermap.org/geo/1.0/direct?q={fixed}&limit=5&appid={API_KEY}"
        r2 = requests.get(url2).json()
        if not r2:
            return None, None
        return r2[0]["lat"], r2[0]["lon"]

    return r[0]["lat"], r[0]["lon"]


# 3. RAINFALL API (PRECIPITATION TILE)
import requests
API_KEY_WAPI = "d243e111150c4739a80135517252711"

def get_rain_forecast(lat, lon, api_key):
    """
    Trả về 5 giá trị:
    - rain_now (mm/h)
    - rain_1h
    - rain_3h
    - rain_6h
    - rain_12h
    Tất cả đều từ WeatherAPI forecast, 100% theo thời gian thật.
    """

    url = (
        f"https://api.weatherapi.com/v1/forecast.json"
        f"?key={API_KEY_WAPI}&q={lat},{lon}&hours=12"
    )

    try:
        js = requests.get(url).json()
    except Exception as e:
        print("Error calling WeatherAPI:", e)
        return (0.0, 0.0, 0.0, 0.0, 0.0)

    # 1) Mưa hiện tại
    rain_now = js.get("current", {}).get("precip_mm", 0.0)

    # 2) List giờ tiếp theo
    hours = js.get("forecast", {}).get("forecastday", [{}])[0].get("hour", [])

    # Đảm bảo đủ 12 giờ
    def safe_precip(index):
        if index < len(hours):
            return hours[index].get("precip_mm", 0.0)
        return 0.0

    rain_1h  = safe_precip(1)   # giờ thứ 1
    rain_3h  = safe_precip(3)
    rain_6h  = safe_precip(6)
    rain_12h = safe_precip(12)

    return (
        float(rain_now),
        float(rain_1h),
        float(rain_3h),
        float(rain_6h),
        float(rain_12h)
    )

# STREAMLIT UI
st.set_page_config(page_title="AI Cảnh Báo Ngập Lụt", layout="wide")
st.title("🌧 AI Cảnh Báo Ngập Lụt — Realtime Weather + Segmentation")

st.write("Dự báo ngập tương lai dựa trên ảnh, segmentation và dữ liệu mưa thời gian thực.")


# LOCATION INPUT
st.subheader("📍 Chọn vị trí lấy dữ liệu thời tiết")

mode = st.radio("Chọn cách nhập:", ["Nhập tên tỉnh (gợi ý tự động)", "Nhập tọa độ"])

lat, lon = None, None

if mode == "Nhập tên tỉnh (gợi ý tự động)":
    user_input = st.text_input("Nhập tên tỉnh/thành phố (VD: Quy Nhon, Ho Chi Minh City):")

    if user_input:
        suggestions = suggest_locations(user_input)
        if len(suggestions) == 0:
            st.error("❌ Không tìm thấy địa điểm phù hợp.")
        else:
            chosen = st.selectbox("Gợi ý:", suggestions)
            lat, lon = get_coordinates(chosen)
            if lat:
                st.success(f"📌 Tọa độ: {lat}, {lon}")

else:
    lat = st.number_input("Latitude:", format="%.6f")
    lon = st.number_input("Longitude:", format="%.6f")

if lat is None or lon is None:
    st.stop()


# WEATHER: RADAR RAIN
st.subheader("🌧 Lượng mưa hiện tại:")
rain_now, rain_1h, rain_3h, rain_6h, rain_12h = get_rain_forecast(lat, lon, API_KEY_WAPI)
st.metric("Rain Radar (mm/h)", f"{rain_now:.2f}")
st.subheader("🌧 Lượng mưa dự báo:")
st.write(f"Trong 1 giờ: {rain_1h:.2f} mm/h")
st.write(f"Trong 3 giờ: {rain_3h:.2f} mm/h")
st.write(f"Trong 6 giờ: {rain_6h:.2f} mm/h")
st.write(f"Trong 12 giờ: {rain_12h:.2f} mm/h")

# IMAGE UPLOAD
uploaded = st.file_uploader("📸 Tải ảnh để phân tích ngập", type=["jpg","jpeg","png"])

if uploaded:
    img = Image.open(uploaded).convert("RGB")

    with NamedTemporaryFile(delete=False, suffix=".png") as tmp:
        img.save(tmp.name)

        # segmentation
        seg_result = predict_flood_level(tmp.name)

        # feature extraction
        ratios, enc = extract_features(tmp.name)

    # UI hiển thị segmentation
    seg_img = seg_result["segmentation"]
    water_ratio = seg_result["water_ratio"]
    level_now = seg_result["flood_text"]

    colA, colB = st.columns(2)
    with colA:
        st.image(img, caption="Ảnh gốc", use_container_width=True)
    with colB:
        st.image(seg_img, caption=f"Segmentation — Ngập: {water_ratio:.1%}", use_container_width=True)

    st.write(f"### 🌊 Mức ngập hiện tại: **{level_now}**")

    semantic = np.array([
        ratios["water_ratio"],
        ratios["building_flooded_ratio"],
        ratios["road_flooded_ratio"],
        ratios["overall_flood_ratio"],
        ratios["tree_ratio"],
        ratios["vehicle_ratio"],
        ratios["pool_ratio"],
        ratios["grass_ratio"],
        rain_now,
        rain_1h,
        rain_3h,
        rain_6h,
        rain_12h,
    ])

    X_input = np.concatenate([semantic, enc])[None, :]
    future_vec = predictor_model.predict(X_input)[0]  # [flood_1h, flood_3h, flood_6h, flood_12h]

    st.subheader("⏱ Chọn mốc thời gian dự báo")
    option = st.radio(
        "Mốc dự báo:",
        ["1 giờ", "3 giờ", "6 giờ", "12 giờ"],
        horizontal=True
    )

    index_map = {"1 giờ": 0, "3 giờ": 1, "6 giờ": 2, "12 giờ": 3}
    future_ratio = float(future_vec[index_map[option]])

    if future_ratio < 0.05:
        future_text = "Không ngập"
    elif future_ratio < 0.15:
        future_text = "Ngập nhẹ"
    elif future_ratio < 0.35:
        future_text = "Ngập vừa"
    else:
        future_text = "Ngập nặng"

    st.metric(f"Mức ngập sau {option}: ", future_text, f"{future_ratio*100:.1f}%")

    if future_ratio >= 0.35:
        st.error("⚠ Cảnh báo ngập nặng!")
    elif future_ratio >= 0.15:
        st.warning("⚠ Nguy cơ ngập — nên theo dõi.")
    else:
        st.success("✔ An toàn — Nguy cơ thấp.")

Overwriting /content/drive/MyDrive/CODE/KHKT/app.py


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

!cloudflared --version

--2025-11-28 06:42:31--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-11-28 06:42:31--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-28T07%3A24%3A14Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

## Run Streamlit

In [ ]:
%cd /content/drive/MyDrive/CODE/KHKT
!streamlit run app.py & cloudflared tunnel --url http://localhost:8501

/content/drive/MyDrive/CODE/KHKT
2025-11-28T06:42:37Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-28T06:42:37Z INF Requesting new quick Tunnel on trycloudflare.com...


2025-11-28T06:42:40Z INF +--------------------------------------------------------------------------------------------+
2025-11-28T06:42:40Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-28T06:42:40Z INF |  https://office-emp